In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties



# Load data

In [2]:
df=pd.read_csv('./submission_sample/submission_sample_travelTime.csv',encoding='utf-8')
traje=pd.read_csv('./phase1_test/trajectories_test_phase1_table5.csv',encoding='utf-8')

In [3]:
df.head(2)

,intersection_id,tollgate_id,time_window,avg_travel_time
0,A,2,"[2016-10-18 08:00:00,2016-10-18 08:20:00)",681.30
1,A,2,"[2016-10-18 08:20:00,2016-10-18 08:40:00)",128.11


In [4]:
traje.head(2)

,intersection_id,tollgate_id,vehicle_id,starting_time,travel_seq,travel_time
0,A,2,1026631,2016-10-18 06:00:14,110#2016-10-18 06:00:14#7.65;123#2016-10-18 06...,27.54
1,C,3,1017312,2016-10-18 06:00:28,115#2016-10-18 06:00:28#9.35;102#2016-10-18 06...,139.53


In [5]:
time_window_new=df['time_window'].str.lstrip('[')
time_window_new=time_window_new.str.rstrip(')')
time_window_new=time_window_new.str.split(',',expand=True)
time_window_new
df['time_window_start']=pd.to_datetime(time_window_new[0])
df['time_window_end']=pd.to_datetime(time_window_new[1])
df.head(5)

,intersection_id,tollgate_id,time_window,avg_travel_time,time_window_start,time_window_end
0,A,2,"[2016-10-18 08:00:00,2016-10-18 08:20:00)",681.30,2016-10-18 08:00:00,2016-10-18 08:20:00
1,A,2,"[2016-10-18 08:20:00,2016-10-18 08:40:00)",128.11,2016-10-18 08:20:00,2016-10-18 08:40:00
2,A,2,"[2016-10-18 08:40:00,2016-10-18 09:00:00)",176.23,2016-10-18 08:40:00,2016-10-18 09:00:00
3,A,2,"[2016-10-18 09:00:00,2016-10-18 09:20:00)",749.09,2016-10-18 09:00:00,2016-10-18 09:20:00
4,A,2,"[2016-10-18 09:20:00,2016-10-18 09:40:00)",37.22,2016-10-18 09:20:00,2016-10-18 09:40:00


In [6]:
from datetime import datetime
from datetime import timedelta
h2_delta=pd.to_datetime('2019-9-4 02:00:00') - pd.to_datetime('2019-9-4 00:00:00')
h2_delta

Timedelta('0 days 02:00:00')

In [7]:
df['time_window_2h_start']=df['time_window_start']-h2_delta
df.head(2)

,intersection_id,tollgate_id,time_window,avg_travel_time,time_window_start,time_window_end,time_window_2h_start
0,A,2,"[2016-10-18 08:00:00,2016-10-18 08:20:00)",681.30,2016-10-18 08:00:00,2016-10-18 08:20:00,2016-10-18 06:00:00
1,A,2,"[2016-10-18 08:20:00,2016-10-18 08:40:00)",128.11,2016-10-18 08:20:00,2016-10-18 08:40:00,2016-10-18 06:20:00


In [8]:
#Load road data
links=pd.read_csv('./road/links_table3.csv',encoding='utf-8')
road=pd.read_csv('./road/routes_table 4.csv',encoding='utf-8')
road['link_list']=road_link=road['link_seq'].str.split(',')
road['link_num']=road['link_list'].map(lambda x: len(x))
road['road_length']=0
links['link_id']=links['link_id'].astype('int')
links.head(2)

,link_id,length,width,lanes,in_top,out_top,lane_width
0,100,58,3,1,105,111,3
1,101,84,3,1,116,121,3


In [9]:
road.head(2)

,intersection_id,tollgate_id,link_seq,link_list,link_num,road_length
0,A,2,"110,123,107,108,120,117","[110, 123, 107, 108, 120, 117]",6,0
1,A,3,"110,123,107,108,119,114,118,122","[110, 123, 107, 108, 119, 114, 118, 122]",8,0


In [10]:
#Compute road length
i=0
for index,row in road.iterrows():
    link_list=getattr(row,'link_list')
    road_length=0
    
    for linkid in link_list:
        road_length+=int(links[links['link_id']==int(linkid)]['length'].values)
    road.loc[i,['road_length']]=road_length
    
    i+=1
    
road.head(5)

,intersection_id,tollgate_id,link_seq,link_list,link_num,road_length
0,A,2,"110,123,107,108,120,117","[110, 123, 107, 108, 120, 117]",6,384
1,A,3,"110,123,107,108,119,114,118,122","[110, 123, 107, 108, 119, 114, 118, 122]",8,852
2,B,1,"105,100,111,103,116,101,121,106,113","[105, 100, 111, 103, 116, 101, 121, 106, 113]",9,821
3,B,3,"105,100,111,103,122","[105, 100, 111, 103, 122]",5,477
4,C,1,"115,102,109,104,112,111,103,116,101,121,106,113","[115, 102, 109, 104, 112, 111, 103, 116, 101, ...",12,1550


In [11]:
df.shape

(504, 7)

In [12]:
# merge data
df=pd.merge(df, road, how='left', on=['intersection_id','tollgate_id'])

In [13]:
df.head(2)

,intersection_id,tollgate_id,time_window,avg_travel_time,time_window_start,time_window_end,time_window_2h_start,link_seq,link_list,link_num,road_length
0,A,2,"[2016-10-18 08:00:00,2016-10-18 08:20:00)",681.30,2016-10-18 08:00:00,2016-10-18 08:20:00,2016-10-18 06:00:00,"110,123,107,108,120,117","[110, 123, 107, 108, 120, 117]",6,384
1,A,2,"[2016-10-18 08:20:00,2016-10-18 08:40:00)",128.11,2016-10-18 08:20:00,2016-10-18 08:40:00,2016-10-18 06:20:00,"110,123,107,108,120,117","[110, 123, 107, 108, 120, 117]",6,384


In [14]:
#transform data type into datetime
traje['starting_time']=pd.to_datetime(traje['starting_time'])
traje.head(5)

,intersection_id,tollgate_id,vehicle_id,starting_time,travel_seq,travel_time
0,A,2,1026631,2016-10-18 06:00:14,110#2016-10-18 06:00:14#7.65;123#2016-10-18 06...,27.54
1,C,3,1017312,2016-10-18 06:00:28,115#2016-10-18 06:00:28#9.35;102#2016-10-18 06...,139.53
2,A,2,1044671,2016-10-18 06:03:07,110#2016-10-18 06:03:07#7.22;123#2016-10-18 06...,26.01
3,A,3,1000017,2016-10-18 06:04:37,110#2016-10-18 06:04:37#7.39;123#2016-10-18 06...,78.40
4,A,2,1002850,2016-10-18 06:10:17,110#2016-10-18 06:10:17#14.33;123#2016-10-18 0...,71.00


# Previous 2-hour Features

In [15]:
#route_car_number,route_avg_speed,route_avg_traveltime,route_total_traveltime

i=0
df['route_car_number']=0
df['route_avg_speed']=0
df['route_avg_traveltime']=0
df['route_total_traveltime']=0
for index,row in df[['intersection_id','tollgate_id','time_window_2h_start','time_window_start']].iterrows():
    #print(getattr(row,'intersection_id'))
    intersection_id=getattr(row,'intersection_id')
    tollgate_id=getattr(row,'tollgate_id')
    window_start=getattr(row,'time_window_2h_start')
    window_end=getattr(row,'time_window_start')
    #window_start=pd.to_datetime(getattr(row,'time_window_2h_start'))
    #window_end=pd.to_datetime(getattr(row,'time_window_start'))
    
    temp_traje=traje[(traje.intersection_id==intersection_id)&(traje.tollgate_id==tollgate_id)
                     &(traje.starting_time>window_start) & (traje.starting_time<window_end)]
    print('-----------',i,'\n')
    print(temp_traje.head(5))
    temp_traje2=temp_traje.groupby(['intersection_id','tollgate_id'])['travel_time'].agg({'route_car_number':'count','route_total_traveltime':'sum'}).reset_index()
    
       
    if not temp_traje2.empty:
        df.loc[i,['route_car_number']]=int(temp_traje2.loc[0,['route_car_number']])
        df.loc[i,['route_avg_traveltime']]=int(temp_traje2.loc[0,['route_total_traveltime']])/int(temp_traje2.loc[0,['route_car_number']])
        df.loc[i,['route_total_traveltime']]=int(temp_traje2.loc[0,['route_total_traveltime']])
        
    
    
    i+=1

print(i,'ENDENDEND')
    

----------- 0 

   intersection_id  tollgate_id  vehicle_id       starting_time  \
0                A            2     1026631 2016-10-18 06:00:14   
2                A            2     1044671 2016-10-18 06:03:07   
4                A            2     1002850 2016-10-18 06:10:17   
5                A            2     1019077 2016-10-18 06:14:19   
11               A            2     1003226 2016-10-18 06:16:24   

                                           travel_seq  travel_time  
0   110#2016-10-18 06:00:14#7.65;123#2016-10-18 06...        27.54  
2   110#2016-10-18 06:03:07#7.22;123#2016-10-18 06...        26.01  
4   110#2016-10-18 06:10:17#14.33;123#2016-10-18 0...        71.00  
5   110#2016-10-18 06:14:19#5.46;123#2016-10-18 06...        20.27  
11  110#2016-10-18 06:16:24#10.06;123#2016-10-18 0...        60.44  
----------- 1 

   intersection_id  tollgate_id  vehicle_id       starting_time  \
16               A            2     1018937 2016-10-18 06:23:54   
17               

/Users/liuxueling/anaconda2/envs/python35/lib/python3.5/site-packages/ipykernel_launcher.py:21: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version


----------- 15 

    intersection_id  tollgate_id  vehicle_id       starting_time  \
707               A            2     1005141 2016-10-20 07:04:59   
710               A            2     1023591 2016-10-20 07:05:53   
714               A            2     1025135 2016-10-20 07:13:15   
716               A            2     1027138 2016-10-20 07:13:42   
717               A            2     1000687 2016-10-20 07:13:47   

                                            travel_seq  travel_time  
707  110#2016-10-20 07:04:59#9.69;123#2016-10-20 07...        40.59  
710  110#2016-10-20 07:05:53#21.54;123#2016-10-20 0...        71.77  
714  110#2016-10-20 07:13:15#19.96;123#2016-10-20 0...       160.26  
716  110#2016-10-20 07:13:42#8.85;123#2016-10-20 07...        31.05  
717  110#2016-10-20 07:13:47#16.69;123#2016-10-20 0...       118.71  
----------- 16 

    intersection_id  tollgate_id  vehicle_id       starting_time  \
732               A            2     1017141 2016-10-20 07:25:25   
7

----------- 30 

     intersection_id  tollgate_id  vehicle_id       starting_time  \
1681               A            2     1002447 2016-10-23 06:27:35   
1682               A            2     1027406 2016-10-23 06:27:40   
1686               A            2     1005366 2016-10-23 06:34:33   
1689               A            2     1042750 2016-10-23 06:42:38   
1695               A            2     1016971 2016-10-23 06:49:00   

                                             travel_seq  travel_time  
1681  110#2016-10-23 06:27:35#15.10;123#2016-10-23 0...        52.84  
1682  110#2016-10-23 06:27:40#28.03;123#2016-10-23 0...        88.73  
1686  110#2016-10-23 06:34:33#9.63;123#2016-10-23 06...        65.37  
1689  110#2016-10-23 06:42:38#8.13;123#2016-10-23 06...        63.75  
1695  110#2016-10-23 06:49:00#33.24;123#2016-10-23 0...       109.77  
----------- 31 

     intersection_id  tollgate_id  vehicle_id       starting_time  \
1681               A            2     1002447 2016-10-23

----------- 45 

   intersection_id  tollgate_id  vehicle_id       starting_time  \
61               A            3     1001399 2016-10-18 07:11:01   
63               A            3     1036644 2016-10-18 07:12:26   
64               A            3     1032361 2016-10-18 07:13:47   
65               A            3     1002834 2016-10-18 07:16:08   
71               A            3     1018013 2016-10-18 07:20:22   

                                           travel_seq  travel_time  
61  110#2016-10-18 07:11:01#15.72;123#2016-10-18 0...       120.02  
63  110#2016-10-18 07:12:26#5.46;123#2016-10-18 07...       116.48  
64  110#2016-10-18 07:13:47#9.84;123#2016-10-18 07...       197.91  
65  110#2016-10-18 07:16:08#10.95;123#2016-10-18 0...       140.18  
71  110#2016-10-18 07:20:22#12.64;123#2016-10-18 0...       125.99  
----------- 46 

   intersection_id  tollgate_id  vehicle_id       starting_time  \
71               A            3     1018013 2016-10-18 07:20:22   
72             

...

    intersection_id  tollgate_id  vehicle_id       starting_time  \
306               C            1     1041081 2016-10-18 16:47:00   
307               C            1     1008064 2016-10-18 16:47:21   
310               C            1     1048376 2016-10-18 16:50:55   

                                            travel_seq  travel_time  
306  115#2016-10-18 16:47:00#9.60;102#2016-10-18 16...       134.17  
307  115#2016-10-18 16:47:21#12.15;102#2016-10-18 1...       200.21  
310  115#2016-10-18 16:50:55#10.72;102#2016-10-18 1...       254.69  
----------- 426 

    intersection_id  tollgate_id  vehicle_id       starting_time  \
469               C            1     1026550 2016-10-19 15:09:03   
506               C            1     1031660 2016-10-19 15:34:19   
515               C            1     1010287 2016-10-19 15:39:52   
522               C            1     1038035 2016-10-19 15:43:05   
530               C            1     1002476 2016-10-19 15:46:28   

                    

----------- 442 

     intersection_id  tollgate_id  vehicle_id       starting_time  \
1271               C            1     1015995 2016-10-21 16:27:47   
1281               C            1     1024317 2016-10-21 16:31:00   
1282               C            1     1024081 2016-10-21 16:31:08   
1289               C            1     1008706 2016-10-21 16:35:43   
1292               C            1     1020272 2016-10-21 16:36:35   

                                             travel_seq  travel_time  
1271  115#2016-10-21 16:27:47#11.04;102#2016-10-21 1...       277.72  
1281  115#2016-10-21 16:31:00#10.15;102#2016-10-21 1...       236.02  
1282  115#2016-10-21 16:31:08#9.73;102#2016-10-21 16...       263.12  
1289  115#2016-10-21 16:35:43#11.07;102#2016-10-21 1...       141.87  
1292  115#2016-10-21 16:36:35#7.59;102#2016-10-21 16...       116.59  
----------- 443 

     intersection_id  tollgate_id  vehicle_id       starting_time  \
1302               C            1     1000876 2016-10-

----------- 458 

     intersection_id  tollgate_id  vehicle_id       starting_time  \
2238               C            1     1005923 2016-10-24 15:51:46   
2243               C            1     1003983 2016-10-24 15:57:15   
2255               C            1     1035017 2016-10-24 16:06:47   
2267               C            1     1029144 2016-10-24 16:13:11   
2279               C            1     1043843 2016-10-24 16:19:13   

                                             travel_seq  travel_time  
2238  115#2016-10-24 15:51:46#8.77;102#2016-10-24 15...       235.04  
2243  115#2016-10-24 15:57:15#6.22;102#2016-10-24 15...       116.69  
2255  115#2016-10-24 16:06:47#8.65;102#2016-10-24 16...       168.08  
2267  115#2016-10-24 16:13:11#7.68;102#2016-10-24 16...       197.66  
2279  115#2016-10-24 16:19:13#8.56;102#2016-10-24 16...       179.34  
----------- 459 

     intersection_id  tollgate_id  vehicle_id       starting_time  \
2255               C            1     1035017 2016-10-

----------- 473 

    intersection_id  tollgate_id  vehicle_id       starting_time  \
637               C            3     1030429 2016-10-19 16:53:56   

                                            travel_seq  travel_time  
637  115#2016-10-19 16:53:56#10.00;102#2016-10-19 1...        177.3  
----------- 474 

    intersection_id  tollgate_id  vehicle_id       starting_time  \
781               C            3     1042533 2016-10-20 15:02:56   
794               C            3     1046939 2016-10-20 15:11:31   
811               C            3     1022776 2016-10-20 15:22:53   
855               C            3     1004467 2016-10-20 15:48:10   
860               C            3     1038957 2016-10-20 15:49:33   

                                            travel_seq  travel_time  
781  115#2016-10-20 15:02:56#9.01;102#2016-10-20 15...       164.90  
794  115#2016-10-20 15:11:31#9.65;102#2016-10-20 15...       131.11  
811  115#2016-10-20 15:22:53#10.94;102#2016-10-20 1...       142.73 

----------- 488 

     intersection_id  tollgate_id  vehicle_id       starting_time  \
1534               C            3     1032095 2016-10-22 15:49:04   
1545               C            3     1024277 2016-10-22 15:58:47   
1572               C            3     1017998 2016-10-22 16:11:48   
1573               C            3     1017910 2016-10-22 16:12:04   
1577               C            3     1013266 2016-10-22 16:12:41   

                                             travel_seq  travel_time  
1534  115#2016-10-22 15:49:04#10.85;102#2016-10-22 1...       216.50  
1545  115#2016-10-22 15:58:47#127.87;102#2016-10-22 ...       372.63  
1572  115#2016-10-22 16:11:48#12.11;102#2016-10-22 1...       211.50  
1573  115#2016-10-22 16:12:04#8.66;102#2016-10-22 16...       240.42  
1577  115#2016-10-22 16:12:41#11.02;102#2016-10-22 1...       219.20  
----------- 489 

     intersection_id  tollgate_id  vehicle_id       starting_time  \
1572               C            3     1017998 2016-10-

In [16]:
df.head(3)

,intersection_id,tollgate_id,time_window,avg_travel_time,time_window_start,time_window_end,time_window_2h_start,link_seq,link_list,link_num,road_length,route_car_number,route_avg_speed,route_avg_traveltime,route_total_traveltime
0,A,2,"[2016-10-18 08:00:00,2016-10-18 08:20:00)",681.30,2016-10-18 08:00:00,2016-10-18 08:20:00,2016-10-18 06:00:00,"110,123,107,108,120,117","[110, 123, 107, 108, 120, 117]",6,384,42,0,54.571429,2292
1,A,2,"[2016-10-18 08:20:00,2016-10-18 08:40:00)",128.11,2016-10-18 08:20:00,2016-10-18 08:40:00,2016-10-18 06:20:00,"110,123,107,108,120,117","[110, 123, 107, 108, 120, 117]",6,384,35,0,57.257143,2004
2,A,2,"[2016-10-18 08:40:00,2016-10-18 09:00:00)",176.23,2016-10-18 08:40:00,2016-10-18 09:00:00,2016-10-18 06:40:00,"110,123,107,108,120,117","[110, 123, 107, 108, 120, 117]",6,384,29,0,60.068966,1742


In [17]:
df['route_avg_speed']=df['road_length']/df['route_avg_traveltime']
df.head(5)

,intersection_id,tollgate_id,time_window,avg_travel_time,time_window_start,time_window_end,time_window_2h_start,link_seq,link_list,link_num,road_length,route_car_number,route_avg_speed,route_avg_traveltime,route_total_traveltime
0,A,2,"[2016-10-18 08:00:00,2016-10-18 08:20:00)",681.30,2016-10-18 08:00:00,2016-10-18 08:20:00,2016-10-18 06:00:00,"110,123,107,108,120,117","[110, 123, 107, 108, 120, 117]",6,384,42,7.036649,54.571429,2292
1,A,2,"[2016-10-18 08:20:00,2016-10-18 08:40:00)",128.11,2016-10-18 08:20:00,2016-10-18 08:40:00,2016-10-18 06:20:00,"110,123,107,108,120,117","[110, 123, 107, 108, 120, 117]",6,384,35,6.706587,57.257143,2004
2,A,2,"[2016-10-18 08:40:00,2016-10-18 09:00:00)",176.23,2016-10-18 08:40:00,2016-10-18 09:00:00,2016-10-18 06:40:00,"110,123,107,108,120,117","[110, 123, 107, 108, 120, 117]",6,384,29,6.392652,60.068966,1742
3,A,2,"[2016-10-18 09:00:00,2016-10-18 09:20:00)",749.09,2016-10-18 09:00:00,2016-10-18 09:20:00,2016-10-18 07:00:00,"110,123,107,108,120,117","[110, 123, 107, 108, 120, 117]",6,384,22,6.672986,57.545455,1266
4,A,2,"[2016-10-18 09:20:00,2016-10-18 09:40:00)",37.22,2016-10-18 09:20:00,2016-10-18 09:40:00,2016-10-18 07:20:00,"110,123,107,108,120,117","[110, 123, 107, 108, 120, 117]",6,384,16,6.467368,59.375000,950


In [18]:
# writer = pd.ExcelWriter('./processed_data/2h_route_avg_test.xlsx',engine='xlsxwriter')
# df.to_csv(writer)
# writer.save()